### Fist, authenticate to google and import librabies

In [ ]:
import ee
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sys
import os
import glob
from tqdm.auto import tqdm

In [ ]:
# mount the google drive
from google.colab import drive
drive.mount('/content/G_drive')

Mounted at /content/G_drive


In [ ]:
# change the current working directory
cwd = '/content/G_drive/MyDrive/Colab Notebooks/Paper_3/Earth_Engine_codes/Step_1_UNET_predict_future_urbanization'
os.chdir(cwd)

# cp GEE authorize tokens
!mkdir /root/.config/earthengine/
!cp '../../../Google_Earth_Engine_tokes/credentials' /root/.config/earthengine/credentials 

# authenticate earthengine api
ee.Initialize()

In [ ]:
# append path so to load custom modules
import sys
sys.path.append('./torch_modules')

# load global variables
from Global_Variables import *
from helper_funcs import export_sample, parse_tfrecord, to_tuple, get_dataset
from helper_funcs import TF_to_npy,eval_model,export_img_bufferd_tf

In [ ]:
# might need reload modules as the code changes
%load_ext autoreload
%autoreload 2

In [ ]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)


# gpu info
!nvidia-smi -L

2.8.2
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# report the simulation years
proj_yr_selected

['1990_1992', '2017_2019']

In [ ]:
# The base names for train_eval data
train_base = f'train_{proj_yr_selected[0]}_prj_{proj_yr_selected[1]}_patch'
eval_base = f'eval_{proj_yr_selected[0]}_prj_{proj_yr_selected[1]}_patch'

### Then, difine the basic parameter/path-to-data

In [ ]:
# import urban-img
urban = ee.Image("users/wangjinzhulala/China_built_up/06_temporal_corrected_classification/Mosaic_only_forward_China_huadong_sample_ensemble")
region = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [ ]:
# Specify the kernel that chop image into smaller tiles.
list_2d = np.ones((KERNEL_SIZE,KERNEL_SIZE))
kernel  = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, list_2d.tolist())

In [ ]:
# create the training patch
train_data = []
    
# get the time
t0 = proj_yr_selected[0]
t1 = proj_yr_selected[1]

# get the built-up layer
t0_img,t1_img = [urban.gte(year_img_val_dict[yr]).rename(f'built_up_t{i}') for i,yr in enumerate(proj_yr_selected)]

# import DEM and SLOPE
DEM = ee.Image("USGS/SRTMGL1_003")
SLOPE = ee.Terrain.slope(DEM)

# stack all bands for later tranin/eval
# IMPORTANT: img.toFloat() is necessary!
img_train = t0_img.addBands(DEM).addBands(SLOPE).addBands(t1_img).select(FEATURES_train).toFloat()  
img_eval = img_train

# the img for prediont to the future
img_pred   = t1_img.addBands(DEM).addBands(SLOPE).rename(FEATURES_train[:-1]).toFloat()

# convert img to neighboorhood-array
arrays_train = img_train.neighborhoodToArray(kernel)
arrays_eval = img_eval.neighborhoodToArray(kernel)

# add patch to a dict
train_data.append([t0,t1,arrays_train,arrays_eval,img_pred])

In [ ]:
train_data = pd.DataFrame(train_data,columns = ['t0','t1','arrays_train','arrays_eval','img_pred'])
train_data

,t0,t1,arrays_train,arrays_eval,img_pred
0,1990_1992,2017_2019,"ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."


#### Export TF-record sampels to G-Drive

In [ ]:
# split the export into patch and shards to avoide "capacity exceed"
patch_n = 20
shard_per_patch = 50

# export the training and eval samples
for sample_type in ["train","eval"]:
  # get the training sample
  for idx,row in train_data.iterrows():
    # export samples
    if sample_type == 'train':  
      img = row['arrays_train']
    else:
      img = row['arrays_eval']
    export_sample(img,
            region,
            proj_yr_selected,
            patch_n,
            shard_per_patch,
            export_type=sample_type)  

train_1990_1992_2017_2019_patch_000
train_1990_1992_2017_2019_patch_001
train_1990_1992_2017_2019_patch_002
train_1990_1992_2017_2019_patch_003
train_1990_1992_2017_2019_patch_004
train_1990_1992_2017_2019_patch_005
train_1990_1992_2017_2019_patch_006
train_1990_1992_2017_2019_patch_007
train_1990_1992_2017_2019_patch_008
train_1990_1992_2017_2019_patch_009
train_1990_1992_2017_2019_patch_010
train_1990_1992_2017_2019_patch_011
train_1990_1992_2017_2019_patch_012
train_1990_1992_2017_2019_patch_013
train_1990_1992_2017_2019_patch_014
train_1990_1992_2017_2019_patch_015
train_1990_1992_2017_2019_patch_016
train_1990_1992_2017_2019_patch_017
train_1990_1992_2017_2019_patch_018
train_1990_1992_2017_2019_patch_019
eval_1990_1992_2017_2019_patch_000
eval_1990_1992_2017_2019_patch_001
eval_1990_1992_2017_2019_patch_002
eval_1990_1992_2017_2019_patch_003
eval_1990_1992_2017_2019_patch_004
eval_1990_1992_2017_2019_patch_005
eval_1990_1992_2017_2019_patch_006
eval_1990_1992_2017_2019_patch_007


In [ ]:
# save the img of research area to TF-record with buffer
img = train_data.iloc[0]['img_pred']
pred_img_base = f'pred_{proj_yr_selected[0]}_{proj_yr_selected[1]}'
export_img_bufferd_tf(img,pred_img_base,KERNEL_BUFFER,region)

In [ ]:
# converting TF-record to torch.tensor
sample_path = '/content/Sample_npy'
if not os.path.exists(sample_path): 
  os.makedirs(sample_path,exist_ok=True)  
TF_to_npy(train_base,eval_base,path=sample_path)

  0%|          | 0/8000 [00:00<?, ?it/s]

./torch_modules/helper_funcs.py:141: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_y_tuple = np.array((x,y))


  0%|          | 0/2000 [00:00<?, ?it/s]

In [ ]:
import zipfile
from zipfile import ZipFile

with ZipFile('./Sample_npy/Sample_npy.zip','w',compression=zipfile.ZIP_DEFLATED) as zip_file:
  for f in tqdm(glob.glob('/content/Sample_npy/*')):
    zip_file.write(f)

  0%|          | 0/9984 [00:00<?, ?it/s]

In [ ]:
# cp sample.py to GCP
from google.colab import auth
auth.authenticate_user()

#### Transfer data to Gcloud

In [ ]:
! gsutil cp '/content/G_drive/MyDrive/Colab Notebooks/Paper_3/Earth_Engine_codes/Step_2_UNET_predict_future_urbanization/Sample_npy/Sample_npy.zip' 'gs://tf-gee/Urban_projection/Sample_npy.zip'

In [ ]:
! gsutil acl ch -u AllUsers:R 'gs://tf-gee/Urban_projection/Sample_npy.zip'

Updated ACL on gs://tf-gee/Urban_projection/Sample_npy.zip
